In [ ]:
#pip install psycopg2
#pip install psycopg2-binary
#pip install sqlalchemy

In [ ]:
import requests
from sqlalchemy import create_engine 
from sqlalchemy.engine import URL
import pandas as pd

# Подключение к постгрес и создание таблиц

In [ ]:
url_object = URL.create(
    "postgresql+psycopg2",     
    username='postgres',     
    password='1234qwe',     
    host='localhost',     
    database='postgres',
    port='5432'
)  
engine = create_engine(url_object)

In [ ]:
query_create_vacancies = """Create table vacancies(
    vacancy_id INT,
    title TEXT,
    min_salary INT,
    max_salary INT, 
    city TEXT,
    work_exp Text,
    company Text,
    employment Text
)"""

In [ ]:
with engine.connect() as conn:
    conn.exec_driver_sql(query_create_vacancies)
    conn.commit()

In [ ]:
query_create_vacancies_queries = """Create table vacancies_queries(
    vacancy_id INT,
    search_id INT
)"""

In [ ]:
with engine.connect() as conn:
    conn.exec_driver_sql(query_create_vacancies_queries)
    conn.commit()

In [ ]:
query_create_queries = """Create table queries(
    search_id SERIAL Primary key,
    search_text Text
)"""

In [ ]:
with engine.connect() as conn:
    conn.exec_driver_sql(query_create_queries)
    conn.commit()

In [ ]:
pd.read_sql_query('select * from information_schema.tables', engine)# проверка, что таблицы успешно созданы

# Парсинг и сохранение в БД

In [ ]:
#список запросов(вакансий) для парсинга
vacancies_to_parse = ['Data Engineer', 'Analyst', 'Data Scientist', 'Business Analyst', 'Python Developer']

In [ ]:
# сохраняем вакансии в БД
pd.DataFrame(vacancies_to_parse, columns=['search_text']).to_sql('queries', con=engine, if_exists='append', index=False)

In [ ]:
# Функция для получения вакансий
def get_vacancies(vacancy, page):
    url = 'https://api.hh.ru/vacancies'
    params = {
        'text': f"{vacancy}",
        'specialization': 1,
        'per_page': 100,
        'page': page
    }

    response = requests.get(url, params=params) 
    return response

In [ ]:
#функция для извлечения нужных данных из json
def parse_vacancy(vacancy_json): 
    vacancy_id = vacancy_json['id']
    title = vacancy_json['name']
    try:
        min_salary = vacancy_json['salary']['from']
    except:
        min_salary = None
    try:
        max_salary = vacancy_json['salary']['to']
    except:
        max_salary = None
    city = vacancy_json['area']['name']
    company = vacancy_json['employer']['name']
    employment = vacancy_json['employment']['name']
    work_exp = vacancy_json['experience']['name']
    
    dct = {'vacancy_id': vacancy_id, 'title': title, 'min_salary': min_salary, 'max_salary': max_salary,
     'city': city, 'work_exp': work_exp, 'company': company, 'employment': employment}
    
    return dct

In [ ]:
for vacancy in vacancies_to_parse:
    #Получаем вакансии на первых 10 страницах по данному запросу
    res_df = pd.DataFrame()
    for page_number in range(1, 11):
        vacansies = get_vacancies(vacancy, page_number)
        for vacancy_json in response.json()['items']:
            res_df = pd.concat([res_df, pd.DataFrame(parse_vacancy(vacancy_json), index=[0])])
    res_df = res_df.drop_duplicates()
    res_df['vacancy_id'] = res_df['vacancy_id'].astype(int)
    
    #получаем список всех сохраненных вакансий, сохраняем только новые
    saved_vacancies = pd.read_sql_query('select vacancy_id from vacancies', engine).vacancy_id.unique()
    res_df.query('vacancy_id not in @saved_vac').to_sql('vacancies', con=engine, if_exists='append', index=False)
    
    #получаем search_id для данного запроса, сохраняем все связи между вакансиями и запросом в таблице vacancies_queries
    queries = pd.read_sql_query('select * from queries', engine)
    search_id = queries.query('search_text == @vacancy')['search_id'].values[0]
    res_df['search_id'] = search_id
    res_df[['vacancy_id', 'search_id']].to_sql('vacancies_queries', con=engine, if_exists='append', index=False)


